In [85]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download
download('stopwords')
download('omw-1.4')
import seaborn as sns
import matplotlib.pyplot as plt
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saiga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Saiga\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [68]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [9]:
sarc = pd.read_csv('datasets/sentiment/sarcasm/SARC/test-balanced.csv',
                    sep='\t',
                    header=None,
                    usecols=[0,1]).rename(columns={0:'label',1:'text'})

In [49]:
sarc.head()

,label,text
0,0,Your mom
1,0,"I'm curious, how much for the White Photons?"
2,0,Khloe Kardashian effect
3,0,Knowledge is power!
4,0,Get the Prussians (Germans) to pay for a statu...


In [79]:
#Tweet Eval : Emotion dection
#Labels and text  are in separate  text files
eval_emotion_train = pd.DataFrame()
eval_emotion_train['text']= pd.read_table('datasets/sentiment/emotion/tweetEval/datasets/train_text.txt',header=None)

In [142]:
remove_emojis(eval_emotion_train.loc[32,'text'])

"@user 'shit' doesn't even begin to describe these fiery little demons straight from hell  ;) "

In [110]:
corpus[32]

['shit',
 'doesnt',
 'even',
 'begin',
 'describe',
 'fiery',
 'little',
 'demon',
 'straight',
 'hell',
 '🌝',
 '🌚']

In [130]:
re.compile(r'(.)\1{2,}', re.IGNORECASE).sub(r'\1', "jaa tuuurlijk jij mag daa doen")

'jaa turlijk jij mag daa doen'

## Draft : text preprocessing


In [147]:
#Preprocessing functions
#Use a tweet tokenizer for the datasets containing tweets

def remove_punctuation(text,is_tweet=False):
    #Remove all punctuation
    if is_tweet:
        punctuation = "!\"$%&'()*+,-?—.../:;<=>[\]^_`{|}~“”‘’•°"
        text = ''.join(char for char in text if char not in punctuation)
    else :    
        punctuation = "!\"$%&'()*+,-?.../:;<=>[\]^_`{|}~“”‘’•°#@"  
        text = ''.join(char for char in text if char not in punctuation)
    return text

def remove_repeated_letters(text):
    #If the same letters appears consecutively 3 times or more, reduce it to 1 occurence
    text = re.compile(r'(.)\1{2,}', re.IGNORECASE).sub(r'\1',text)
    return text

def remove_emojis(text):
    #Remove all emojis 
    text = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE).sub(r'',text)          
    return text
            
def remove_digits(text):
    #Remove all numbers 
    text = [word for word in text if not word.isdigit()]
    return text

def remove_stopwords(text,stopwords):
    #Remove stopwords
    text = [word for word in text if word not in stopwords]
    return text


def remove_urls(text):
    #Remove urls 
    text = [word for word in text if 'http' not in word]
    return text

def lemmatize(text,lem):
    #Lemmatize text 
    text = [lem.lemmatize(word) for word in text]
    return text 

def remove_hashtags_mentions(text):
    #Remove hashtags and mentions (for tweets)
    text = [word for word in text if word[0] not in ["#","@"]]
    return text

In [148]:
%%time
tt = TweetTokenizer()
corpus = eval_emotion_train['text'].fillna('').str.lower().to_list()[0:50]
corpus = [remove_punctuation(text,True) for text in corpus]
corpus = [remove_repeated_letters(text) for text in corpus]
corpus = [remove_emojis(text) for text in corpus]
#corpus = [word_tokenize(text) for text in corpus]
corpus = [tt.tokenize(text) for text in corpus]
corpus = [remove_digits(text) for text in corpus]
corpus = [remove_hashtags_mentions(text) for text in corpus]
corpus = [remove_stopwords(text,sw) for text in corpus]
corpus = [remove_urls(text) for text in corpus]
corpus = [lemmatize(text,lemmatizer) for text in corpus]

Wall time: 23.9 ms
